Copied from the end of visualise_how_params_change_sed, will probably deprecate

## Can we recover the original parameters?

In [ ]:
theta_to_recover = theta_agn_extinction

In [ ]:
model_spectrum, model_photometry, _ = model.sed(theta_to_recover, obs, sps)  # trigger fsps calculation, takes a couple of minutes - then cached

In [ ]:
def make_synthetic_obs_from_model(real_obs, maggies, snr=10.):
    synthetic_obs = {}
    synthetic_obs["filters"] =  real_obs['filters']
    synthetic_obs["maggies"] = maggies
    synthetic_obs['maggies_unc'] = maggies / snr

    synthetic_obs["phot_mask"] = np.array([True for _ in synthetic_obs['filters']])
    synthetic_obs["phot_wave"] = np.array([f.wave_effective for f in synthetic_obs["filters"]])
    synthetic_obs["wavelength"] = None
    synthetic_obs["spectrum"] = None
    synthetic_obs['unc'] = None
    synthetic_obs['mask'] = None
    synthetic_obs = fix_obs(synthetic_obs)
    return synthetic_obs


In [ ]:
synthetic_obs = make_synthetic_obs_from_model(obs, model_photometry)

### Max Likelihood (starting from correct values, so unhelpful!)

In [ ]:
# theta_best, time_elapsed = main.fit_galaxy(run_params, synthetic_obs, model, sps)

### MCMC

In [ ]:
samples, mcmc_time_elapsed = main.mcmc_galaxy(run_params, synthetic_obs, model, sps, initial_theta=None, test=False)

In [ ]:
synthetic_obs = build_synthetic_obs_from_model(obs, model_photometry, snr=10.)

In [ ]:
visualise.visualise_obs(synthetic_obs)

In [ ]:
name = 'inference_to_recover_theta'
output_dir = '/home/mike/repos/agnfinder/results'

In [ ]:
sample_loc = os.path.join(output_dir, '{}_mcmc_samples.h5py'.format(name))
main.save_samples(samples, model, sample_loc)
corner_loc = os.path.join(output_dir, '{}_mcmc_corner.png'.format(name))
main.save_corner(samples[int(len(samples)/2):], model, corner_loc)  # nested sampling has no burn-in phase, early samples are bad

In [ ]:
theta_to_recover

In [ ]:
import corner

In [ ]:
for index in range(len(theta_to_recover)):
    print('Name: {}'.format(model.free_params[index]))
    print('True value: {:3.1E}'.format(theta_to_recover[index]))
    low, med, up = corner.quantile(samples[:, index], [.1, .5, .9])
    print('Estimate: {:3.1E} (min {:3.2E}, max {:3.3E})'.format(low, med, up))
    print('\n')
    

In [ ]:
traces_loc = os.path.join(output_dir, '{}_mcmc_traces.png'.format(name))
main.save_sed_traces(samples[-2000:], synthetic_obs, model, sps, traces_loc)


In [ ]:
plt.loglog(sps.wavelengths, sps.extincted_quasar_flux)
_, spectra, _ = sps.get_galaxy_spectrum()
plt.loglog(sps.wavelengths, spectra)

In [ ]:
len(sps.wavelengths)

In [ ]:
sps.extincted_quasar_flux.sum() / model_spectrum.sum()

In [ ]:
len(sps.extincted_quasar_flux)

In [ ]:
len(model_spectrum)